# Preparando o Ambiente

Utilizamos a versão 3.9.12 do _python_, apresentando o nosso estudo a partir da interface jupyter[^1], usando anaconda[^2] como gerenciador de pacotes. As bibliotecas utilizadas e suas respectivas versões estão descritas a seguir:

[^1]: https://jupyter.org/
[^2]: https://www.anaconda.com/

In [1]:
!cat ../requirements.txt

requests==2.28.1
pandas==1.4.2
datatable
tqdm==4.64.0

In [8]:
conda install pip
pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Escolha do Dataset

Buscamos uma base de dados de procedência confiável, que possuísse também dados históricos para possibilitar uma análise temporal mais apurada e não apenas um olhar mais fechado na última amostra. As bases selecionadas foram provenientes do Censo de Educação Superior,[^3] fornecido pelo INEP, , usando a tabela `Microdados | Censo da Educação Superior`, disponível em  `https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_{year}.zip}`, em que o `year` corresponde ao ano alvo para cada coleta.  
Levamos em consideração dados coletados no período entre 2010 e 2019 e, a partir disso, fizemos uma agregação de todas as tabelas em um único arquivo que chamamos de `cursos`.

Abaixo, está o esse código de coleta dos microdados, usando uma função auxiliar para automatizar o processo.

[^3]: https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/censo-da-educacao-superior/resultados

In [3]:
import os
import pandas as pd
import datatable as dt
import requests, warnings
from tqdm import tqdm

In [2]:
def download_raw_data(
    url_base,
    from_year,
    to_year,
    output_dir=''
  ):
  os.makedirs(os.path.dirname(output_dir), exist_ok=True)
  years = range(from_year, to_year + 1)
  n_years = to_year - from_year + 1
  for y in tqdm(years, total=n_years):
    fname = f'censo_{y}.zip'
    export = output_dir + fname
    url = url_base.format(year = y)
    try:
      with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        r = requests.get(
          url, verify = False)
      with open(export, 'wb') as f:
        f.write(r.content)
      print(f'Success to download year: {y}')
    except:
      print(f'Failed to download year: {y} from {url}')


In [3]:
url_base = '/'.join([
  'https://download.inep.gov.br',
  'microdados',
  'microdados_censo_da_educacao_superior_{year}.zip'])

from_year = 2010
to_year = 2019

output_dir = '../data/raw/'
download_raw_data(
  url_base,
  from_year,
  to_year,
  output_dir
)

 10%|█         | 1/10 [00:03<00:28,  3.21s/it]

Success to download year: 2010


 20%|██        | 2/10 [00:06<00:26,  3.35s/it]

Success to download year: 2011


 30%|███       | 3/10 [00:12<00:31,  4.52s/it]

...

Success to download year: 2017


 90%|█████████ | 9/10 [00:42<00:05,  5.34s/it]

Success to download year: 2018


100%|██████████| 10/10 [00:54<00:00,  5.43s/it]

Success to download year: 2019


# Datasets Complementares

Datasets que contém informações que serão úteis para nossa análise.

## Latitude & Longitude dos Municípios

https://github.com/kelvins/Municipios-Brasileiros/blob/main/csv/municipios.csv

In [7]:
url_mun = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'
df_mun = pd.read_csv(url_mun)
df_mun

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo
...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,0,29,3971,74,America/Sao_Paulo
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo
5567,3557154,Zacarias,-21.05060,-50.0552,0,35,2973,18,America/Sao_Paulo
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo


In [8]:
path_mun = '../data/municipios.csv.zip'
df_mun.to_csv(path_mun, index=False)